In [3]:
import utils
import json
from mapboxfetcher import MapBoxFetcher
import numpy as np
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import PIL
import pickle
from treemodel import TreeModel
import imagefunctions
import os.path
from shapely.geometry import Polygon
from keras.models import load_model
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
nn_model = load_model('NNModelOne128One256One128Layer.h5')
# with open('TrainedModelManualZoom16.p', 'rb') as f:
#     ada_model = pickle.load(f)
tree_model = TreeModel(nn_model)

In [5]:
def squareCoordinatesToGeoJSON(coordinates):
    if isinstance(coordinates, np.ndarray):
        coordinates = coordinates.tolist()
    feature = {
        'type': 'Feature',
        'properties': {},
        'geometry': {
            'type': 'Polygon',
            'coordinates': [[
                coordinates[0][::-1],
                coordinates[1][::-1],
                coordinates[2][::-1],
                coordinates[3][::-1],
                coordinates[0][::-1]
            ]]
        }
    }
    return json.dumps(feature)
def polygonFromSquareCoordinates(coordinates):
    return Polygon([(c[1], c[0]) for c in coordinates])
def getTileCorners(x, y, zoom):
    ul = utils.num2deg(x, y, zoom)
    ur = utils.num2deg(x+1, y, zoom)
    lr = utils.num2deg(x+1, y+1, zoom)
    ll = utils.num2deg(x, y+1, zoom)
    return [ul, ur, lr, ll]
def getTileGeoJSON(x, y, zoom):
    ul, ur, lr, ll = getTileCorners(x, y, zoom)
    return squareCoordinatesToGeoJSON([ul, ur, lr, ll])
def splitTile(x, y, zoom, splits = 8):
    ul, ur, lr, ll = getTileCorners(x, y, zoom)
    lat_diff = (ul[0] - ll[0]) / splits
    long_diff = (ur[1] - ul[1]) / splits
    out = np.ndarray((splits, splits, 4, 2))
    for i in range(splits):
        for j in range(splits):
            out[i,j,:,:] = [
                [ul[0] - i*lat_diff, ul[1] + j*long_diff],
                [ul[0] - i*lat_diff, ul[1] + (j+1)*long_diff],
                [ul[0] - (i+1)*lat_diff, ul[1] + (j+1)*long_diff],
                [ul[0] - (i+1)*lat_diff, ul[1] + j*long_diff],
            ]
    return out

In [17]:
geojson = json.load(open('Vaud.geojson'))
polygon = utils.polygon_union(geojson['features'])
bounds = polygon.bounds
zoom = 16

In [ ]:
lat, lon = bounds[3], bounds[0]
start_numx, start_numy = utils.deg2num(lat, lon, zoom)
numx, numy = start_numx, start_numy

tile_count = 0
total_tile_count = 0
tile_x_count = 0
tile_y_count = 0
fetcher = MapBoxFetcher()

while lat >= bounds[1]:
    while lon <= bounds[2]:
        c_poly = polygonFromSquareCoordinates(getTileCorners(numx, numy, zoom))
        if polygon.intersects(c_poly):
            total_tile_count += 1
        tile_count += 1

        numx += 1
        lon = utils.num2deg(numx, numy, zoom)[1]
    lon = bounds[0]
    numx = start_numx
    numy += 1
    tile_y_count += 1
    lat = utils.num2deg(numx, numy, zoom)[0]
tile_x_count = tile_count // tile_y_count
    
print('Polygon contains {} tiles at zoom level {} with tile resolution {}x{}'.format(
    total_tile_count,
    zoom,
    tile_y_count,
    tile_x_count
))

In [14]:
zoom = 16
lat, lon = bounds[3], bounds[0]
start_numx, start_numy = utils.deg2num(lat, lon, zoom)
numx, numy = start_numx, start_numy

fetcher = MapBoxFetcher()

data = []

while lat >= bounds[1]:
    read_numxs = []
    added = False
    if os.path.isfile('Data/Predictions-Y{}.p'.format(numy)):
        with open('Data/Predictions-Y{}.p'.format(numy), 'rb') as f:
            data = pickle.load(f)
            for c in data:
                read_numxs.append(c['x'])
    while lon <= bounds[2]:
        if numx in read_numxs:
            numx += 1
            lon = utils.num2deg(numx, numy, zoom)[1]
            continue
        c_poly = polygonFromSquareCoordinates(getTileCorners(numx, numy, zoom))
        if polygon.intersects(c_poly):
            sat_img = fetcher.satellite(numx, numy, zoom, hq=True)
            data.append({
                'x': numx,
                'y': numy,
                'pred': imagefunctions.threshold(imagefunctions.filter(tree_model.predict_proba(sat_img.array())))
            })
            added = True
        numx += 1
        lon = utils.num2deg(numx, numy, zoom)[1]
    if added:
        pickle.dump(data, open('Data/Predictions-Y{}.p'.format(numy), 'wb'))
    print('Fetched and predicted satellite row {} with {} images along X'.format(numy, len(data)))
    data = []
        
    lon = bounds[0]
    numx = start_numx
    numy += 1
    lat = utils.num2deg(numx, numy, zoom)[0]

Fetched and predicted satellite row 23219 with 83 images along X
Fetched and predicted satellite row 23220 with 82 images along X
Fetched and predicted satellite row 23221 with 81 images along X
Fetched and predicted satellite row 23222 with 81 images along X
Fetched and predicted satellite row 23223 with 81 images along X
Fetched and predicted satellite row 23224 with 79 images along X
Fetched and predicted satellite row 23225 with 81 images along X
Fetched and predicted satellite row 23226 with 79 images along X
Fetched and predicted satellite row 23227 with 74 images along X
Fetched and predicted satellite row 23228 with 73 images along X
Fetched and predicted satellite row 23229 with 73 images along X
Fetched and predicted satellite row 23230 with 71 images along X
Fetched and predicted satellite row 23231 with 72 images along X
Fetched and predicted satellite row 23232 with 70 images along X
Fetched and predicted satellite row 23233 with 69 images along X
Fetched and predicted sat